# Model training
In this demo, you will learn how to:
* configure a model
* train a model

## Imports

In [1]:
from pathlib import Path
from paddle.utilities import download_file, unzip

## Preparations
For the demonstration, we need to download a data set to train on.

### Download and extract data set

In [2]:
data_zip_url = "https://uni-duisburg-essen.sciebo.de/s/26xZ3VBcsA0SnC3/download"
data_zip_path = Path("..") / "mpac.zip"
data_root = Path("..") / "data"

download_file(data_zip_url, data_zip_path)
unzip(data_zip_path, data_root)

  0%|▎                                                                                                                                                                          | 555k/260M [00:00<00:46, 5.53MiB/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 260M/260M [00:22<00:00, 11.6MiB/s]


## Configuration
To control the training, we use a configuration file. For this demonstration the configuration File
is `ExampleConfig.yaml`. So let's see what's inside:


In [3]:
print(open("ExampleConfig.yaml").read())

hydra:
  run:
    dir: logs/${hydra:job.config_name}/${now:%Y-%m-%d_%H-%M-%S}
program:
  random_seed: 42
  search_optimum_learning_rate: false
callbacks:
  early_stopping:
    monitor: validation/mAP
    patience: 20
    mode: max
  model_checkpoint:
    monitor: validation/mAP
    mode: max
    filename: '{epoch}-{step}-{val/mAP:.4f}'
  example_detection_monitor:
    score_threshold: 0.5
    do_display_box: true
    do_display_label: false
    do_display_score: true
    do_display_mask: true
    do_display_outlines_only: true
    line_width: 1
    font_size: 16
logging:
  wandb: 
    use: false
trainer:
  fast_dev_run: false
  max_epochs: 3
  gpus: -1
lightning_module:
  learning_rate: 0.005
  drop_lr_on_plateau_patience: 10
  num_classes: 2
  model_kwargs:
    box_detections_per_img: 200
data_module:
  data_root: ../data
  batch_size: 4
  train_subset: training
  val_subset: validation
  initial_cropping_rectangle: null
  class_selector: null
  user_albumentation_train:
    transform

A few notable configuration keys are:
* `hydra:run:dir`: Naming scheme for the log path (includes a time stamp).
* `callbacks:early_stopping`: Early stopping monitors how the mean average precision of the network
on the validation data set evolves and stops, if there is no improvement for
`callbacks:early_stopping:patience` epochs.
* `trainer:max_epochs`: For this demonstration it is set to 3, to have a fast training. However,
usually the value should be much higher (e.g. 300 or even more). Ideally, this number is never
reached and the training is stopped based on the values of `callbacks:early_stopping`.
* `lightning_module:num_classes`: Number of object classes + 1 (for the omnipresent  background
class).
* `lightning_module:model_kwargs:box_detections_per_img`: Maximum number of detected objects per
image. Can be reduced to improve performance and increased for very crowded images.
* `data_module:train_subset`: Name of the training subset.
* `data_module:val_subset`: Name of the validation subset.

## Training
While in this demonstration, we use Jupyter Notebook, _paddle_ is designed to be run from scripts.
Therefore, we just execute the training script and point it to the path where our configuration is
stored and specify the name of our configuration. The resulting logs are stored in the `logs`
folder.


In [4]:
%run ../../train_model.py --config-dir="" --config-name=ExampleConfig

DIPlib -- a quantitative image analysis library
Version 3.3.0 (May 21 2022)
For more information see https://diplib.org
[2022-07-23 09:11:38,870][paddle.training][INFO] - Training with the following config:
program:
  random_seed: 42
  search_optimum_learning_rate: false
callbacks:
  early_stopping:
    monitor: validation/mAP
    patience: 20
    mode: max
  model_checkpoint:
    monitor: validation/mAP
    mode: max
    filename: '{epoch}-{step}-{val/mAP:.4f}'
  example_detection_monitor:
    score_threshold: 0.5
    do_display_box: true
    do_display_label: false
    do_display_score: true
    do_display_mask: true
    do_display_outlines_only: true
    line_width: 1
    font_size: 16
logging:
  wandb:
    use: false
trainer:
  fast_dev_run: false
  max_epochs: 3
  gpus: -1
lightning_module:
  learning_rate: 0.005
  drop_lr_on_plateau_patience: 10
  num_classes: 2
  model_kwargs:
    box_detections_per_img: 200
data_module:
  data_root: ../data
  batch_size: 4
  train_subset: train

Global seed set to 42
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type       | Params
--------------------------------------------------
0 | model              | MaskRCNN   | 43.9 M
1 | validation_metrics | ModuleDict | 0     
--------------------------------------------------
43.7 M    Trainable params
222 K     Non-trainable params
43.9 M    Total params


Epoch 0:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 100/125 [00:57<00:14,  1.75it/s, loss=1.22, v_num=1-38]
Validating: 0it [00:00, ?it/s]
Epoch 1:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 100/125 [00:56<00:14,  1.78it/s, loss=1.09, v_num=1-38]
Validating: 0it [00:00, ?it/s]
Epoch 2:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 100/125 [00:56<00:14,  1.78it/s, loss=1.01, v_num=1-38]
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [01:47<00:00,  1.17it/s, loss=1.01, v_num=1-38]
